In [2]:
print("hello world")

hello world


In [3]:
from langchain_community.retrievers import WikipediaRetriever

In [4]:
retriever=WikipediaRetriever(top_k_results=2,lang="en")

In [5]:
querry="tell me about the difference between ai engineer and ml engineer"

docs=retriever.invoke(querry)

In [6]:
docs

[Document(metadata={'title': 'Explainable artificial intelligence', 'summary': 'Within artificial intelligence (AI), explainable AI (XAI), generally overlapping with interpretable AI or explainable machine learning (XML), is a field of research that explores methods that provide humans with the ability of intellectual oversight over AI algorithms. The main focus is on the reasoning behind the decisions or predictions made by the AI algorithms, to make them more understandable and transparent. This addresses users\' requirement to assess safety and scrutinize the automated decision making in applications. XAI counters the "black box" tendency of machine learning, where even the AI\'s designers cannot explain why it arrived at a specific decision.\nXAI hopes to help users of AI-powered systems perform more effectively by improving their understanding of how those systems reason. XAI may be an implementation of the social right to explanation. Even if there is no such legal right or regul

In [7]:
for i , doc in enumerate(docs):
    print("\n"+"-"*10+f"result {i+1}"+ "-"*10+"\n")
    print(doc.page_content)



----------result 1----------

Within artificial intelligence (AI), explainable AI (XAI), generally overlapping with interpretable AI or explainable machine learning (XML), is a field of research that explores methods that provide humans with the ability of intellectual oversight over AI algorithms. The main focus is on the reasoning behind the decisions or predictions made by the AI algorithms, to make them more understandable and transparent. This addresses users' requirement to assess safety and scrutinize the automated decision making in applications. XAI counters the "black box" tendency of machine learning, where even the AI's designers cannot explain why it arrived at a specific decision.
XAI hopes to help users of AI-powered systems perform more effectively by improving their understanding of how those systems reason. XAI may be an implementation of the social right to explanation. Even if there is no such legal right or regulatory requirement, XAI can improve the user experien

In [8]:
from langchain_community.vectorstores import Chroma
from langchain_core.documents import Document
from langchain_huggingface import HuggingFaceEmbeddings
from dotenv import load_dotenv
load_dotenv()
embedding=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [9]:
documents = [
    Document(page_content="LangChain helps developers build LLM applications easily."),
    Document(page_content="Chroma is a vector database optimized for LLM-based search."),
    Document(page_content="Embeddings convert text into high-dimensional vectors."),
    Document(page_content="OpenAI provides powerful embedding models."),
]

In [10]:
vector_store= Chroma.from_documents(
    documents=documents,
    embedding=embedding,
    collection_name="my_collection"
)

In [11]:
retriever1=vector_store.as_retriever(search_kwargs={"k":2})

In [12]:
querry="what is chroma used for"
result=retriever1.invoke(querry)

In [13]:
result

[Document(metadata={}, page_content='Chroma is a vector database optimized for LLM-based search.'),
 Document(metadata={}, page_content='LangChain helps developers build LLM applications easily.')]

### MMR


In [14]:
docs = [
    Document(page_content="LangChain makes it easy to work with LLMs."),
    Document(page_content="LangChain is used to build LLM based applications."),
    Document(page_content="Chroma is used to store and search document embeddings."),
    Document(page_content="Embeddings are vector representations of text."),
    Document(page_content="MMR helps you get diverse results when doing similarity search."),
    Document(page_content="LangChain supports Chroma, FAISS, Pinecone, and more."),
]

In [15]:
from langchain_community.vectorstores import FAISS

In [16]:
vector_store= Chroma.from_documents(
    documents=docs,
    embedding=embedding,
    collection_name="my_collection"
)

In [17]:
retriever=vector_store.as_retriever(
    search_type="mmr",
    search_kwargs={"k":2,"lambda_mult":0.5}
)

In [18]:
querry="what is langchian ?"
retriever.invoke(querry)

[Document(metadata={}, page_content='LangChain supports Chroma, FAISS, Pinecone, and more.'),
 Document(metadata={}, page_content='Embeddings convert text into high-dimensional vectors.')]

In [19]:
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
llm=HuggingFaceEndpoint(
    repo_id="katanemo/Arch-Router-1.5B",
    task="text-generation"
)
model=ChatHuggingFace(llm=llm)

In [20]:
from langchain_classic.retrievers import MultiQueryRetriever

In [21]:
# Relevant health & wellness documents
all_docs = [
    Document(page_content="Regular walking boosts heart health and can reduce symptoms of depression.", metadata={"source": "H1"}),
    Document(page_content="Consuming leafy greens and fruits helps detox the body and improve longevity.", metadata={"source": "H2"}),
    Document(page_content="Deep sleep is crucial for cellular repair and emotional regulation.", metadata={"source": "H3"}),
    Document(page_content="Mindfulness and controlled breathing lower cortisol and improve mental clarity.", metadata={"source": "H4"}),
    Document(page_content="Drinking sufficient water throughout the day helps maintain metabolism and energy.", metadata={"source": "H5"}),
    Document(page_content="The solar energy system in modern homes helps balance electricity demand.", metadata={"source": "I1"}),
    Document(page_content="Python balances readability with power, making it a popular system design language.", metadata={"source": "I2"}),
    Document(page_content="Photosynthesis enables plants to produce energy by converting sunlight.", metadata={"source": "I3"}),
    Document(page_content="The 2022 FIFA World Cup was held in Qatar and drew global energy and excitement.", metadata={"source": "I4"}),
    Document(page_content="Black holes bend spacetime and store immense gravitational energy.", metadata={"source": "I5"}),]

In [22]:
vector_store=Chroma.from_documents(documents=all_docs,embedding=embedding)

In [23]:
multi_querry=MultiQueryRetriever.from_llm(
    retriever=vector_store.as_retriever(search_kwargs={"k":5}),
    llm=model
)

In [24]:
querry="how to improve energy level and and mantain balance?"

In [25]:
result=multi_querry.invoke(querry)
result

[Document(metadata={'source': 'H4'}, page_content='Mindfulness and controlled breathing lower cortisol and improve mental clarity.'),
 Document(metadata={'source': 'H5'}, page_content='Drinking sufficient water throughout the day helps maintain metabolism and energy.'),
 Document(metadata={'source': 'H1'}, page_content='Regular walking boosts heart health and can reduce symptoms of depression.'),
 Document(metadata={'source': 'H3'}, page_content='Deep sleep is crucial for cellular repair and emotional regulation.'),
 Document(metadata={'source': 'H2'}, page_content='Consuming leafy greens and fruits helps detox the body and improve longevity.'),
 Document(metadata={'source': 'I1'}, page_content='The solar energy system in modern homes helps balance electricity demand.')]

In [37]:
from langchain_classic.retrievers import ContextualCompressionRetriever
from langchain_classic.retrievers.document_compressors import LLMChainExtractor

In [38]:
# Recreate the document objects from the previous data
docs = [
    Document(page_content=(
        """The Grand Canyon is one of the most visited natural wonders in the world.
        Photosynthesis is the process by which green plants convert sunlight into energy.
        Millions of tourists travel to see it every year. The rocks date back millions of years."""
    ), metadata={"source": "Doc1"}),

    Document(page_content=(
        """In medieval Europe, castles were built primarily for defense.
        The chlorophyll in plant cells captures sunlight during photosynthesis.
        Knights wore armor made of metal. Siege weapons were often used to breach castle walls."""
    ), metadata={"source": "Doc2"}),

    Document(page_content=(
        """Basketball was invented by Dr. James Naismith in the late 19th century.
        It was originally played with a soccer ball and peach baskets. NBA is now a global league."""
    ), metadata={"source": "Doc3"}),

    Document(page_content=(
        """The history of cinema began in the late 1800s. Silent films were the earliest form.
        Thomas Edison was among the pioneers. Photosynthesis does not occur in animal cells.
        Modern filmmaking involves complex CGI and sound design."""
    ), metadata={"source": "Doc4"})
]

In [39]:
vector_store=Chroma.from_documents(documents=docs,embedding=embedding)

In [40]:
compressor=LLMChainExtractor.from_llm(model)

In [41]:
base_retriever = vector_store.as_retriever(search_kwargs={"k": 5})

In [42]:
compression_retriever = ContextualCompressionRetriever(
    base_retriever=base_retriever,
    base_compressor=compressor
)

In [43]:
query = "What is photosynthesis?"
compressed_results = compression_retriever.invoke(query)

In [44]:
for i, doc in enumerate(compressed_results):
    print(f"\n--- Result {i+1} ---")
    print(doc.page_content)



--- Result 1 ---
Photosynthesis is the process by which green plants convert sunlight into energy.

--- Result 2 ---
Photosynthesis is the process by which green plants convert sunlight into energy.
